In [1]:
# Preliminaries
from tqdm import tqdm
import math
import random
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

# Visuals and CV2
import cv2

# albumentations for augs
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

#torch
import torch
import timm
import torch
import torch.nn as nn
from torch.nn import Parameter
from torch.nn import functional as F
from torch.utils.data import Dataset,DataLoader

import warnings
warnings.filterwarnings('ignore')

# Configuration

In [2]:
class CFG:
    
    img_size = 512
    batch_size = 12
    seed = 2020
    
    device = 'cuda'
    classes = 11014
    
    scale = 30 
    margin = 0.5

In [3]:
DIM = (512,512)

NUM_WORKERS = 8
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 12
EPOCHS = 20
SEED = 2020

device = torch.device('cuda')

#model_name = 'efficientnet_b3' #efficientnet_b0-b7
num_ch = 512

# Utils

In [4]:
def seed_torch(seed=2020):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_torch(SEED)

In [5]:
def get_valid_transforms():

    return albumentations.Compose(
        [
            albumentations.Resize(DIM[0],DIM[1],always_apply=True),
            albumentations.Normalize(),
        ToTensorV2(p=1.0)
        ]
    )

# Dataset

In [6]:
class ShopeeDataset(Dataset):
    def __init__(self, csv, transforms=None):

        self.csv = csv.reset_index()
        self.augmentations = transforms

    def __len__(self):
        return self.csv.shape[0]

    def __getitem__(self, index):
        row = self.csv.iloc[index]
        
        text = row.title
        
        image = cv2.imread(row.filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.augmentations:
            augmented = self.augmentations(image=image)
            image = augmented['image']       
        
        
        return image,torch.tensor(row.label_group)

In [7]:
class Mish_func(torch.autograd.Function):
    
    """from: https://github.com/tyunist/memory_efficient_mish_swish/blob/master/mish.py"""
    
    @staticmethod
    def forward(ctx, i):
        result = i * torch.tanh(F.softplus(i))
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
  
        v = 1. + i.exp()
        h = v.log() 
        grad_gh = 1./h.cosh().pow_(2) 

        # Note that grad_hv * grad_vx = sigmoid(x)
        #grad_hv = 1./v  
        #grad_vx = i.exp()
        
        grad_hx = i.sigmoid()

        grad_gx = grad_gh *  grad_hx #grad_hv * grad_vx 
        
        grad_f =  torch.tanh(F.softplus(i)) + i * grad_gx 
        
        return grad_output * grad_f 


class Mish(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()
        pass
    def forward(self, input_tensor):
        return Mish_func.apply(input_tensor)


def replace_activations(model, existing_layer, new_layer):
    
    """A function for replacing existing activation layers"""
    
    for name, module in reversed(model._modules.items()):
        if len(list(module.children())) > 0:
            model._modules[name] = replace_activations(module, existing_layer, new_layer)

        if type(module) == existing_layer:
            layer_old = module
            layer_new = new_layer
            model._modules[name] = layer_new
    return model

In [8]:
from functools import reduce

def intersect(*args):
    return reduce(np.intersect1d, args)

def concat(*args):
    return np.unique(np.concatenate(args))

def higher(f,*args):
    res = {}
    keys = np.unique(np.concatenate(args))
    for k in keys: 
        res[k] = np.count_nonzero(np.concatenate(args) == k)
    output_dict = dict(filter(lambda item: item[1] >= f, res.items()))
    
    return np.array(list(output_dict.keys()))

def count(*args):
    res = {}
    keys = np.unique(np.concatenate(args))
    for k in keys: 
        res[k] = np.count_nonzero(np.concatenate(args) == k)
    return res

In [9]:
class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM,self).__init__()
        self.p = nn.Parameter(torch.ones(1)*p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)
        
    def __repr__(self):
        return self.__class__.__name__ + '(' + 'p=' + '{:.4f}'.format(self.p.data.tolist()[0]) + ', ' + 'eps=' + str(self.eps) + ')'

# Model

In [10]:
'''
credit : https://github.com/HuangYG123/CurricularFace/blob/8b2f47318117995aa05490c05b455b113489917e/head/metrics.py#L70
'''

def l2_norm(input, axis = 1):
    norm = torch.norm(input, 2, axis, True)
    output = torch.div(input, norm)

    return output

class CurricularFace(nn.Module):
    def __init__(self, in_features, out_features, s = 30, m = 0.50):
        super(CurricularFace, self).__init__()

        print('Using Curricular Face')

        self.in_features = in_features
        self.out_features = out_features
        self.m = m
        self.s = s
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.threshold = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        self.kernel = nn.Parameter(torch.Tensor(in_features, out_features))
        self.register_buffer('t', torch.zeros(1))
        nn.init.normal_(self.kernel, std=0.01)

    def forward(self, embbedings, label):
        embbedings = l2_norm(embbedings, axis = 1)
        kernel_norm = l2_norm(self.kernel, axis = 0)
        cos_theta = torch.mm(embbedings, kernel_norm)
        cos_theta = cos_theta.clamp(-1, 1)  # for numerical stability
        with torch.no_grad():
            origin_cos = cos_theta.clone()
        target_logit = cos_theta[torch.arange(0, embbedings.size(0)), label].view(-1, 1)

        sin_theta = torch.sqrt(1.0 - torch.pow(target_logit, 2))
        cos_theta_m = target_logit * self.cos_m - sin_theta * self.sin_m #cos(target+margin)
        mask = cos_theta > cos_theta_m
        final_target_logit = torch.where(target_logit > self.threshold, cos_theta_m, target_logit - self.mm)

        hard_example = cos_theta[mask]
        with torch.no_grad():
            self.t = target_logit.mean() * 0.01 + (1 - 0.01) * self.t
        cos_theta[mask] = hard_example * (self.t + hard_example)
        cos_theta.scatter_(1, label.view(-1, 1).long(), final_target_logit)
        output = cos_theta * self.s
        return output, nn.CrossEntropyLoss()(output,label)

In [11]:
class ArcMarginProduct(nn.Module):
    def __init__(self, in_features, out_features, scale=30.0, margin=0.50, easy_margin=False, ls_eps=0.0):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.scale = scale
        self.margin = margin
        self.ls_eps = ls_eps  # label smoothing
        self.weight = nn.Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(margin)
        self.sin_m = math.sin(margin)
        self.th = math.cos(math.pi - margin)
        self.mm = math.sin(math.pi - margin) * margin

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt(1.0 - torch.pow(cosine, 2))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        if self.ls_eps > 0:
            one_hot = (1 - self.ls_eps) * one_hot + self.ls_eps / self.out_features
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
        output *= self.scale

        return output

class ShopeeModel(nn.Module):

    def __init__(
        self,
        n_classes = CFG.classes,
        model_name = None,
        fc_dim = 512,
        margin = CFG.margin,
        scale = CFG.scale,
        use_fc = True,
        pretrained = False):


        super(ShopeeModel,self).__init__()
        print('Building Model Backbone for {} model'.format(model_name))

        self.backbone = timm.create_model(model_name, pretrained=pretrained)

        if model_name == 'resnest101e':
            final_in_features = self.backbone.fc.in_features
            self.backbone.fc = nn.Identity()
            self.backbone.global_pool = nn.Identity()

        elif 'efficientnet' in model_name:
            final_in_features = self.backbone.classifier.in_features
            self.backbone.classifier = nn.Identity()
            self.backbone.global_pool = nn.Identity()
        
        elif model_name == 'eca_nfnet_l0':
            final_in_features = self.backbone.head.fc.in_features
            self.backbone.head.fc = nn.Identity()
            self.backbone.head.global_pool = nn.Identity()

        self.pooling = GeM()

        self.use_fc = use_fc

        self.dropout = nn.Dropout(p=0.0)
        self.fc = nn.Linear(final_in_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        final_in_features = fc_dim

        self.final = CurricularFace(
            final_in_features,
            n_classes,
            s = scale,
            m = margin
        )

    def _init_params(self):
        nn.init.xavier_normal_(self.fc.weight)
        nn.init.constant_(self.fc.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def forward(self, image, label):
        feature = self.extract_feat(image)
        #logits = self.final(feature,label)
        return feature

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone(x)
        x = self.pooling(x).view(batch_size, -1)

        if self.use_fc:
            x = self.dropout(x)
            x = self.fc(x)
            x = self.bn(x)
        return x

In [12]:
def get_model(model_name = None, model_path = None, n_classes = None):
    
    model = ShopeeModel(model_name = model_name)
    if model_name == 'eca_nfnet_l0'or 'tf_efficientnet_b5':
        model = replace_activations(model, torch.nn.SiLU, Mish())
    elif model_name == 'resnest101e':
        model = replace_activations(model, torch.nn.ReLU, Mish())
    model.eval()
    model.load_state_dict(torch.load(model_path))
    model = model.to(CFG.device)
    
    return model 

In [13]:
class EnsembleModel(nn.Module):
    
    def __init__(self):
        super(EnsembleModel,self).__init__()
        
        self.m1 = get_model('eca_nfnet_l0','weight/cface_512x512_nfnet_l0(mish)_gem_20.pt')
        self.m2 = get_model('resnest101e','weight/cface_512x512_resnest(mish)_gem.pt')
        self.m3 = get_model('tf_efficientnet_b5','weight/cface_512x512_effb5(mish)_gem.pt')
        
    def forward(self,img,label):
        
        feat1 = self.m1(img,label)#.cpu().detach().numpy() 
        feat2 = self.m2(img,label)#.cpu().detach().numpy() 
        feat3 = self.m3(img,label)#.cpu().detach().numpy() 
        
        return feat1,feat2,feat3

# Get embeddings

In [14]:
def get_img_emb(data_loader,model,criterion,device):
    model.eval()
    tk0 = tqdm(enumerate(data_loader), total=len(data_loader))
    outs = np.zeros([len(valid), num_ch])
    with torch.no_grad():      
        for i,(bi,d) in enumerate(tk0):
            batch_size = d[0].size()[0]

            image = d[0]
            targets = d[1]

            image = image.to(device)
            targets = targets.to(device)
            # Inference
            output = model(image, targets)
            outs[i*batch_size:i*batch_size+batch_size] = output.cpu().detach().numpy()      
    return outs

# Setup dataloader

In [15]:
from sklearn.model_selection import GroupKFold
df_train = pd.read_csv("input/train_st_fold42.csv")
skf = GroupKFold(5)
df_train['fold'] = -1
for i, (train_idx, valid_idx) in enumerate(skf.split(X=df_train, groups=df_train['label_group'])):
    df_train.loc[valid_idx, 'fold'] = i
train_df = df_train
df_train.tail()

data = df_train
data['filepath'] = data['image'].apply(lambda x: os.path.join('input/train_images', x))
len(data)

34250

In [16]:
fold = 0
train = data[data['fold']!=fold].reset_index(drop=True)
valid = data[data['fold']==fold].reset_index(drop=True)

In [17]:
valid_dataset = ShopeeDataset(
    csv=valid[:100],
    transforms=get_valid_transforms(),
)

valid_loader = torch.utils.data.DataLoader(
    valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    num_workers=NUM_WORKERS,
    shuffle=False,
    pin_memory=True,
    drop_last=False,
)

In [18]:
# Defining Device
device = torch.device("cuda")

# Defining Model for specific fold
model = EnsembleModel()
model = model.to(device)

Building Model Backbone for eca_nfnet_l0 model
Using Curricular Face
Building Model Backbone for resnest101e model
Using Curricular Face
Building Model Backbone for tf_efficientnet_b5 model
Using Curricular Face


In [19]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

from sklearn.preprocessing import normalize
import gc

def get_cv(df, outs):
    thresholds = list(np.arange(0.2, 0.8, 0.1))
    scores = []
    
    # set target
    tmp = df.groupby('label_group').posting_id.agg('unique').to_dict()
    df['target'] = df.label_group.map(tmp)

    # Normalize
    outsn = normalize(outs)

    # to torch
    outsn_torch = torch.from_numpy(outsn).cuda()
    
    # calculate cosine simularity with torch cuda()
    distances = 1 - torch.matmul(outsn_torch, outsn_torch.T).cpu().T
    
    for threshold in thresholds:
        predictions = []
        for k in range(outs.shape[0]):
            idx = np.where(distances[k,] < threshold)[0]
            o = df.iloc[idx].posting_id.values
            predictions.append(o)
        print(np.array(predictions))
        df["preds"] = predictions
        #df['oof'] = df.apply(combine_for_cv,axis=1)
        df['f1'] = df.apply(getMetric("preds"),axis=1)
        score = df['f1'].mean()
        print(f'Our f1 score for threshold {threshold} is {score}')
        scores.append(score)
    thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
    max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
    best_threshold = max_score['thresholds'].values[0]
    best_score = max_score['scores'].values[0]
    print(f'Our best score is {best_score} and has a threshold {best_threshold}')
    gc.collect()
    torch.cuda.empty_cache()

    return best_score

In [20]:
# get embeddings
outs = get_img_emb(valid_loader,model,None,device)

# calculate CV
best = get_cv(valid, outs)

  0%|          | 0/9 [00:01<?, ?it/s]


AttributeError: 'tuple' object has no attribute 'cpu'

# Run CV